In [9]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import ElasticNet

import mlflow
import mlflow.sklearn
from mlflow.models import infer_signature

import warnings
warnings.filterwarnings("ignore")

In [13]:
def eval_metrics(actual, pred):
    rmse = np.sqrt(mean_squared_error(actual, pred))
    mae = mean_absolute_error(actual, pred)
    r2 = r2_score(actual, pred)
    return rmse, mae, r2



warnings.filterwarnings("ignore")
np.random.seed(40)

# Read the wine-quality csv file (make sure you're running this from the root of MLflow!)
wine_path = "wine-quality.csv"
data = pd.read_csv(wine_path)

# Split the data into training and test sets. (0.75, 0.25) split.
train, test = train_test_split(data)

# The predicted column is "quality" which is a scalar from [3, 9]
train_x = train.drop(["quality"], axis=1)
test_x = test.drop(["quality"], axis=1)
train_y = train[["quality"]]
test_y = test[["quality"]]

alpha=0.42
l1_ratio=0.7


mlflow.set_experiment("Wine")


with mlflow.start_run():
    lr = ElasticNet(alpha=alpha, l1_ratio=l1_ratio, random_state=42)
    lr.fit(train_x, train_y)

    predicted_qualities = lr.predict(test_x)

    (rmse, mae, r2) = eval_metrics(test_y, predicted_qualities)

    print("Elasticnet model (alpha=%f, l1_ratio=%f):" % (alpha, l1_ratio))
    print("  RMSE: %s" % rmse)
    print("  MAE: %s" % mae)
    print("  R2: %s" % r2)

    mlflow.log_param("alpha", alpha)
    mlflow.log_param("l1_ratio", l1_ratio)
    mlflow.log_metric("rmse", rmse)
    mlflow.log_metric("r2", r2)
    mlflow.log_metric("mae", mae)
    # Infer the model signature
    signature = infer_signature(train_x, lr.predict(train_x))
    mlflow.sklearn.log_model(
         sk_model=lr, 
         artifact_path="wini_model",
         signature=signature,
         input_example=train_x,
         registered_model_name="wini-model",
                            )

Elasticnet model (alpha=0.420000, l1_ratio=0.700000):
  RMSE: 0.8301929562310988
  MAE: 0.6312807412252281
  R2: 0.1098197370595353


Registered model 'wini-model' already exists. Creating a new version of this model...
2024/11/20 16:03:40 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: wini-model, version 2
Created version '2' of model 'wini-model'.
2024/11/20 16:03:40 INFO mlflow.tracking._tracking_service.client: 🏃 View run melodic-fly-594 at: http://localhost:5000/#/experiments/2/runs/3088a0b90e704b788c94783b6e1341e5.
2024/11/20 16:03:40 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://localhost:5000/#/experiments/2.
